# Data Import and Cleaning 

#### Dataset from here:
#### Column definition from here: https://www2.census.gov/programs-surveys/sipp/data/datasets/2008/l08puw1.sas

In [1]:
import numpy as np
import pandas as pd

In [2]:
input_dict = np.loadtxt('input_dict.txt', dtype = str)
input_dict

array([['SSUSEQ', '1-5'],
       ['SSUID', '6-17'],
       ['SPANEL', '18-21'],
       ...,
       ['LGTKEY', '2328-2335'],
       ['LGTMON', '2336-2337'],
       ['FILLER', '2338-2340']], dtype='<U9')

In [9]:
column_lst = input_dict[:, 0].tolist()

['SSUSEQ',
 'SSUID',
 'SPANEL',
 'SWAVE',
 'SROTATON',
 'SREFMON',
 'RHCALMN',
 'RHCALYR',
 'SHHADID',
 'GVARSTR',
 'GHLFSAM',
 'GRGC',
 'TFIPSST',
 'TMOVRFLG',
 'EOUTCOME',
 'RHNF',
 'RHNFAM',
 'RHNSF',
 'EHREFPER',
 'EHHNUMPP',
 'RHTYPE',
 'WHFNWGT',
 'TMETRO',
 'RHCHANGE',
 'RHNSSR',
 'ETENURE',
 'ATENURE',
 'EPUBHSE',
 'APUBHSE',
 'EGVTRNT',
 'AGVTRNT',
 'TMTHRNT',
 'AMTHRNT',
 'EWRSECT8',
 'AWRSECT8',
 'EUTILYN',
 'AUTILYN',
 'EEGYAST',
 'AEGYAST',
 'EEGYPMT1',
 'EEGYPMT2',
 'EEGYPMT3',
 'AEGYPMT',
 'EEGYAMT',
 'AEGYAMT',
 'EHOTLUNC',
 'AHOTLUNC',
 'RNKLUN',
 'EFREELUN',
 'AFREELUN',
 'EFRERDLN',
 'AFRERDLN',
 'EBRKFST',
 'ABRKFST',
 'RNKBRK',
 'EFREEBRK',
 'AFREEBRK',
 'EFRERDBK',
 'AFRERDBK',
 'RPRGQUES',
 'THEARN',
 'THPRPINC',
 'THTRNINC',
 'THOTHINC',
 'THTOTINC',
 'RHNBRF',
 'RHCBRF',
 'RHMTRF',
 'RHPOV',
 'THPNDIST',
 'THLUMPSM',
 'THNONCSH',
 'THSOCSEC',
 'THSSI',
 'THUNEMP',
 'THVETS',
 'THAFDC',
 'THFDSTP',
 'RFID',
 'RFID2',
 'EFNP',
 'EFREFPER',
 'EFSPOUSE',
 'EFTYPE',

In [7]:
input_dict[:, 1]
# loop through each string and split it at the '-'
# put it in a list of tuples

array(['1-5', '6-17', '18-21', ..., '2328-2335', '2336-2337', '2338-2340'],
      dtype='<U9')

In [ ]:
colspecs=[(0,10),(11,17),(17,20),(20,24)]
names=['DP','PHONETYPE','MARKET','FOLDER'])

data = pd.read_fwf('p08putm1.dat', colspecs, names
data.head()